# Semantic Search + NER Demo

Built following the excellent [blog post by Camille Corti-Georgiou](https://www.elastic.co/search-labs/blog/articles/developing-an-elastic-search-app-with-streamlit-semantic-search-and-named-entity-extraction).


## Import Dependencies


In [51]:
import csv
import getpass
import os
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from getpass import getpass

from elasticsearch import Elasticsearch, exceptions
from elasticsearch.helpers import parallel_bulk
from kaggle.api.kaggle_api_extended import KaggleApi
from tqdm.auto import tqdm  # auto selects notebook GUI if in Jupyter

## Download dataset from Kaggle


In [4]:
# Configure the Kaggle API
os.environ["KAGGLE_USERNAME"] = getpass.getpass("Enter your Kaggle username:")
os.environ["KAGGLE_KEY"] = getpass.getpass("Enter your Kaggle key:")

# Initialize API
api = KaggleApi()
api.authenticate()

# Define the dataset path
dataset = "gpreda/bbc-news"

# Define download path
download_path = "../data"  # Adjust this path as needed

# Download dataset
api.dataset_download_files(dataset, path=download_path, unzip=True)

print(f"Dataset downloaded to: {download_path}")

Dataset downloaded to: ../data


## Use Docker to Upload and Start the Custom NER ML Model

```sh
docker run -it --rm docker.elastic.co/eland/eland:latest \
    eland_import_hub_model \
        --cloud-id $CLOUD_ID \
        --es-api-key $API_KEY \
        --hub-model-id "elastic/distilbert-base-uncased-finetuned-conll03-english" \
        --task-type ner \
        --start
```


## Set up Elasticsearch Client


In [6]:
# Create the client instance
es_client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=getpass("Elastic Cloud ID: "),
    api_key=getpass("Elastic Api Key: "),
)

print(es_client.info())

{'name': 'instance-0000000030', 'cluster_name': '0a22c2712d2e41cdb96896213f6ebd0d', 'cluster_uuid': 'rh-W7PoJRkW5zpO9L3-pYg', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


## Download and Deploy ELSER Model

This code will download the ELSER model. This will take a few minutes to complete. Use the following command to check the status of the model download.


In [7]:
model_id = ".elser_model_2_linux-x86_64"

# delete model if already downloaded and deployed
try:
    es_client.ml.delete_trained_model(model_id=model_id, force=True)
    print("Model deleted successfully, We will proceed with creating one")

except exceptions.NotFoundError:
    print("Model doesn't exist, but We will proceed with creating one")

# Creates the ELSER model configuration. Automatically downloads the model if it doesn't exist.
es_client.ml.put_trained_model(model_id=model_id, input={"field_names": ["text_field"]})

while True:
    status = es_client.ml.get_trained_models(
        model_id=model_id, include="definition_status"
    )

    if status["trained_model_configs"][0]["fully_defined"]:
        print("ELSER Model is downloaded and ready to be deployed.")
        break

    else:
        print("ELSER Model is downloaded but not ready to be deployed.")
    time.sleep(5)

/var/folders/9d/6ddqcctn1p53z_4xwltwpwc40000gn/T/ipykernel_15051/4165779150.py:3: ElasticsearchWarning: The default [remove_binary] value of 'false' is deprecated and will be set to 'true' in a future release. Set [remove_binary] explicitly to 'true' or 'false' to ensure no behavior change.
  es_client.ml.delete_trained_model(model_id=".elser_model_2", force=True)


Model deleted successfully, We will proceed with creating one
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded but not ready to be deployed.
ELSER Model is downloaded and ready to be deployed.


Once the model is downloaded, we can deploy the model in our ML node. Use the following command to deploy the model.


In [66]:
# Start trained model deployment if not already deployed
es_client.ml.start_trained_model_deployment(
    model_id=".elser_model_2_linux-x86_64",
    wait_for="starting",
    queue_capacity=10000,
    number_of_allocations=7,
    threads_per_allocation=2,
)

while True:
    status = es_client.ml.get_trained_models_stats(
        model_id=".elser_model_2_linux-x86_64",
    )

    if status["trained_model_stats"][0]["deployment_stats"]["state"] == "started":
        print("ELSER Model has been successfully deployed.")
        break

    else:
        print("ELSER Model is currently being deployed.")
    time.sleep(5)

ELSER Model is currently being deployed.
ELSER Model is currently being deployed.
ELSER Model has been successfully deployed.


## Create Elasticsearch Index & Ingest Pipeline


## Define Ingest Pipeline processors, and index mappings

First we need to define the json representing the ingest pipeline processors, and the index mapping


In [36]:
es_news_pipeline_processors = [
    {"date": {"field": "pubDate", "formats": ["EEE, dd MMM yyyy HH:mm:ss zzz"]}},
    {
        "inference": {
            "model_id": ".elser_model_2",
            "input_output": [
                {"input_field": "title", "output_field": "ml-elser-title"}
            ],
            "description": 'Runs .elser_model_2 and stores resulting tokens in "ml-elser-title"',
        }
    },
    {
        "inference": {
            "model_id": ".elser_model_2",
            "input_output": [
                {"input_field": "description", "output_field": "ml-elser-description"}
            ],
            "description": 'Runs .elser_model_2 and stores resulting tokens in "ml-elser-description"',
        }
    },
    {
        "inference": {
            "model_id": "elastic__distilbert-base-uncased-finetuned-conll03-english",
            "target_field": "ml.ner",
            "field_map": {"title": "text_field"},
        }
    },
    {
        "script": {
            "lang": "painless",
            "if": "return ctx['ml']['ner'].containsKey('entities')",
            "source": "Map tags = new HashMap(); for (item in ctx['ml']['ner']['entities']) { if (!tags.containsKey(item.class_name)) tags[item.class_name] = new HashSet(); tags[item.class_name].add(item.entity);} ctx['tags'] = tags;",
        }
    },
]

In [43]:
es_news_index_mapping = {
    "_source": {"excludes": ["content_embedding"]},
    "properties": {
        "@timestamp": {"type": "date"},
        "@version": {"type": "keyword", "ignore_above": 256},
        "description": {"type": "text"},
        "ml": {
            "properties": {
                "ner": {
                    "properties": {
                        "entities": {
                            "properties": {
                                "class_name": {"type": "keyword", "ignore_above": 256},
                                "class_probability": {"type": "float"},
                                "end_pos": {"type": "long"},
                                "entity": {"type": "keyword", "ignore_above": 256},
                                "start_pos": {"type": "long"},
                            }
                        },
                        "model_id": {"type": "keyword", "ignore_above": 256},
                        "predicted_value": {"type": "keyword", "ignore_above": 256},
                    }
                }
            }
        },
        "ml-elser-description": {"type": "sparse_vector"},
        "ml-elser-title": {"type": "sparse_vector"},
        "pubDate": {
            "type": "date",
            "format": "EEE, dd MMM yyyy HH:mm:ss z",
            "ignore_malformed": True,
        },
        "tags": {
            "properties": {
                "LOC": {"type": "keyword", "ignore_above": 256},
                "MISC": {"type": "keyword", "ignore_above": 256},
                "ORG": {"type": "keyword", "ignore_above": 256},
                "PER": {"type": "keyword", "ignore_above": 256},
            }
        },
        "title": {"type": "text"},
        "url": {"type": "keyword", "ignore_above": 256},
    },
}

now, to create the ingest pipeline and search index...


In [53]:
es_client.ingest.put_pipeline(
    id="search-demos-bbc.semantic.ner-pipeline",
    description="Ingest pipeline for Semantic Ner Search Demo using BBC dataset",
    processors=es_news_pipeline_processors,
)

ObjectApiResponse({'acknowledged': True})

In [77]:
es_index = "search-demos-bbc.semantic.ner"

es_client.indices.delete(index="search-demos-bbc.semantic.ner", ignore_unavailable=True)
es_client.indices.create(
    index=es_index,
    settings={"index": {"default_pipeline": "search-demos-bbc.semantic.ner-pipeline"}},
    mappings=es_news_index_mapping,
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'search-demos-bbc.semantic.ner'})

## Process the CSV data and index into Elasticsearch


In [78]:
# Path to your dataset file
dataset_path = "../data/bbc_news.csv"

# Read the CSV file into a dict
data = []
with open(dataset_path, mode="r", encoding="utf-8") as file:
    # Create a DictReader object
    csv_reader = csv.DictReader(file)

    # Iterate over the rows in the CSV file
    for row in csv_reader:
        data.append(row)

# Now 'data' is a list of dictionaries, where each dictionary represents a row from the CSV file
print(data[0:5])  # Print the first row to verify

[{'title': 'Ukraine: Angry Zelensky vows to punish Russian atrocities', 'pubDate': 'Mon, 07 Mar 2022 08:01:56 GMT', 'guid': 'https://www.bbc.co.uk/news/world-europe-60638042', 'link': 'https://www.bbc.co.uk/news/world-europe-60638042?at_medium=RSS&at_campaign=KARANGA', 'description': 'The Ukrainian president says the country will not forgive or forget those who murder its civilians.'}, {'title': 'War in Ukraine: Taking cover in a town under attack', 'pubDate': 'Sun, 06 Mar 2022 22:49:58 GMT', 'guid': 'https://www.bbc.co.uk/news/world-europe-60641873', 'link': 'https://www.bbc.co.uk/news/world-europe-60641873?at_medium=RSS&at_campaign=KARANGA', 'description': 'Jeremy Bowen was on the frontline in Irpin, as residents came under Russian fire while trying to flee.'}, {'title': "Ukraine war 'catastrophic for global food'", 'pubDate': 'Mon, 07 Mar 2022 00:14:42 GMT', 'guid': 'https://www.bbc.co.uk/news/business-60623941', 'link': 'https://www.bbc.co.uk/news/business-60623941?at_medium=RSS&at

### Some helper functions for a smooth ingest experience


In [79]:
def generate_actions(documents: list):
    """
    Generates actions for bulk ingest from a list of dictionaries.

    Args:
        documents (list): A list of dictionaries, where each dictionary is a document.

    Yields:
        dict: A dictionary representing an action for bulk ingest.
    """
    for document in documents:
        yield document


def send_to_elasticsearch(
    es_client: Elasticsearch,
    es_index: str,
    documents: list,
    chunk_size: int = 500,
) -> str:
    """
    Uploads data to an Elasticsearch index using parallel processing.

    Args:
        es_client (Elasticsearch): The Elasticsearch client.
        es_index (str): The name of the Elasticsearch index.
        documents (list): A list of dictionaries to index.
        chunk_size (int): The number of documents to send in each bulk request.

    Returns:
        str: A message indicating the number of successfully indexed documents.
    """
    num_documents = len(documents)
    progress = tqdm(unit="docs", total=num_documents)
    successes = 0

    try:
        for ok, info in parallel_bulk(
            client=es_client,
            index=es_index,
            actions=generate_actions(documents),
            chunk_size=chunk_size,
            request_timeout=360,
        ):
            progress.update(1)
            progress.refresh()

            if ok:
                successes += 1
            else:
                print(f"A document failed to index: {info['index']['error']}")

    except Exception as e:
        print("An Elasticsearch error occurred:", e)
        if hasattr(e, "errors"):
            for error_detail in e.errors:
                print(error_detail)

    return f"Indexed {successes}/{num_documents} documents."

### The actual data ingest


In [80]:
num_workers = 8
thread_count = 16
chunk_size = 1000
batch_size = 500

# print("Indexing documents...")

# result = send_to_elasticsearch(
#     es_client=es_client,
#     es_index=es_index,
#     documents=data,
#     chunk_size=chunk_size,
# )

# print(result)

In [81]:
def upload_in_batches(es_client, es_index, documents, batch_size=500):
    total_docs = len(documents)
    start_index = 0

    while start_index < total_docs:
        # Calculate the end index for the current batch, ensuring it doesn't exceed total_docs
        end_index = min(start_index + batch_size, total_docs)
        current_batch = documents[start_index:end_index]

        # Upload the current batch
        result = send_to_elasticsearch(
            es_client=es_client,
            es_index=es_index,
            documents=current_batch,
            chunk_size=batch_size,  # This ensures parallel_bulk processes in chunks of 500 as well
        )

        # Optionally, print/log the result of each batch upload
        print(result)

        # Update the start index for the next batch
        start_index += batch_size


# Use the function
upload_in_batches(es_client, es_index, data, batch_size=batch_size)

  0%|          | 0/500 [00:00<?, ?docs/s]

Indexed 500/500 documents.


  0%|          | 0/500 [00:00<?, ?docs/s]

Indexed 500/500 documents.


  0%|          | 0/500 [00:00<?, ?docs/s]

...

Indexed 500/500 documents.


  0%|          | 0/500 [00:00<?, ?docs/s]

Indexed 500/500 documents.


  0%|          | 0/134 [00:00<?, ?docs/s]

Indexed 134/134 documents.
